## Script responsável por gerar embeddings com Sentence-BERT (SBERT)

O **Sentence-BERT (SBERT)** é uma extensão do modelo BERT, adaptado para gerar embeddings de sentenças altamente eficientes. Desenvolvido para superar as limitações de BERT em tarefas de similaridade semântica e recuperação de informações, o SBERT combina a robustez de BERT com uma arquitetura otimizada para calcular representações vetoriais diretamente comparáveis entre sentenças. Ele utiliza redes *Siamese* ou *Triplet* durante o treinamento, permitindo calcular similaridades semânticas entre pares de sentenças de forma muito mais rápida e precisa. O SBERT é amplamente utilizado em tarefas como busca semântica, agrupamento de textos e classificação, oferecendo alta performance com embeddings prontos para serem utilizados em modelos de aprendizado supervisionado ou não supervisionado. Sua versatilidade e eficiência fazem dele uma escolha popular para aplicações em linguística computacional e aprendizado de máquina.

### Faz o embedding das colunas => coluna output
- **Texto** => `SBERT_Embeddings1`
- **TextoTratado** => `SBERT_Embeddings2`
- **TextoTratadoSemNomes** => `SBERT_Embeddings3`
- **TextoTratadoSemNomesLemmatized** => `SBERT_Embeddings4`

**OBS**: Utiliza o arquivo **./saida/04_1_DOU2_portarias_mgi_tratado_NER_lematizado_embeddingsUSE.parquet**, caso não exista, executar o 04_1_Embedding_USE.ipynb



## Bibliotecas

In [1]:
# Bibliotecas padrão do Python
import time  # Para medir e monitorar o tempo de execução

# Bibliotecas externas para manipulação de dados e cálculos
import pandas as pd  # Manipulação de dados em tabelas (DataFrames)
import numpy as np  # Trabalhos com arrays e operações matemáticas

# Bibliotecas externas para processamento de linguagem natural
from sentence_transformers import SentenceTransformer  # Gerar embeddings de sentenças

In [2]:
douItem = 2
secao = f'Secao0{douItem}'

# Lê o arquivo Parquet
df_portarias_mgi = pd.read_parquet(f'./saida/04_1_DOU{secao}_portarias_mgi_tratado_NER_lematizado_embeddingsUSE.parquet', engine='pyarrow')  # ou engine='fastparquet'

# Exibe as primeiras linhas do DataFrame
df_portarias_mgi.shape


(19258, 43)

In [3]:
df_portarias_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,Texto_sem_nomes,Nomes,TextoTratadoSemNomes,TextoTratadoSemNomes_TotalPalavras,TextoTratadoSemNomesLemmatized,TextoTratadoSemNomesLemmatized_TotalPalavras,USE_Embeddings1,USE_Embeddings2,USE_Embeddings3,USE_Embeddings4
0,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"sergio eduardo arbulu, anderson pereira silvy,...","sgp / sedgg / me nº 529 , 17 janeiro 2023 secr...",268,sgp Sedgg nº 529 17 janeiro 2023 secretario ge...,182,"[-0.050729595, -0.01408702, 0.053643852, 0.055...","[-0.044771984, -0.056401458, 0.033406682, 0.05...","[-0.049094215, -0.05746024, 0.043547586, 0.054...","[-0.04014247, -0.06005167, -0.046900157, 0.053..."
1,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"eliane araujo franklin, eduardo arbulu","portaria sgp / sedgg / me nº 655 , 23 janeiro ...",189,portar sgp Sedgg nº 655 23 janeiro 2023 secret...,127,"[-0.048080508, 0.013370708, 0.05068045, 0.0507...","[-0.046608303, -0.04633815, -0.0065669254, 0.0...","[-0.04858907, -0.053217582, 0.03618075, 0.0538...","[-0.030576829, -0.0599243, -0.040472377, 0.061..."
2,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,eduardo arbulu,"sgp / sedgg / me nº 461 , 16 janeiro 2023 secr...",197,sgp Sedgg nº 461 16 janeiro 2023 secretario ge...,132,"[-0.043791097, 0.013420352, 0.05320274, 0.0461...","[-0.045567505, -0.05334681, 0.0037366129, 0.05...","[-0.046353806, -0.055992313, 0.03370651, 0.049...","[-0.019732608, -0.061047755, -0.034037452, 0.0..."
3,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,eduardo arbulu,"sgp / sedgg / me nº 606 , 19 janeiro 2023 secr...",187,sgp Sedgg nº 606 19 janeiro 2023 secretario ge...,134,"[-0.045023933, -0.0064749066, 0.03732383, 0.04...","[-0.051662467, -0.059400998, 0.011962159, 0.05...","[-0.05032127, -0.06039041, 0.026458807, 0.0492...","[-0.047755465, -0.06160219, -0.036089916, 0.04..."
4,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"roberto jorge ferreira, termino exercicio, edu...","sgp / sedgg / me nº 517 , 17 janeiro 2023 secr...",177,sgp Sedgg nº 517 17 janeiro 2023 secretario ge...,122,"[-0.05169902, -0.025740517, 0.04578067, 0.0534...","[-0.059112933, -0.061392505, 0.025555337, 0.05...","[-0.060482606, -0.06389035, 0.03746268, 0.0550...","[-0.049283903, -0.063080825, -0.030478371, 0.0..."


In [ ]:
# Carregar o modelo SBERT
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # Modelo rápido e eficiente

# Função para criar embeddings
def create_embeddings(texts):
    """
    Gera embeddings para uma lista de textos usando o Sentence-BERT (SBERT).
    
    Args:
        texts (list): Lista de textos para os quais os embeddings serão gerados.
    
    Returns:
        np.ndarray: Matriz de embeddings, onde cada linha é o embedding de um texto.
    """
    return sbert_model.encode(texts, convert_to_numpy=True)

# Garantir que as colunas de texto não tenham valores nulos
columns_to_process = ['Texto', 'TextoTratado', 'TextoTratadoSemNomes', 'TextoTratadoSemNomesLemmatized']
for column in columns_to_process:
    df_portarias_mgi[column] = df_portarias_mgi[column].fillna("")

# Função para processar embeddings em blocos para várias colunas
def process_embeddings_in_blocks(df, columns, output_columns, block_size=200):
    """
    Processa o DataFrame em blocos para gerar embeddings de várias colunas.

    Args:
        df (pd.DataFrame): DataFrame contendo as colunas de texto para processar.
        columns (list): Lista de nomes das colunas de texto para gerar embeddings.
        output_columns (list): Lista de nomes das colunas de saída para armazenar os embeddings.
        block_size (int): Tamanho de cada bloco a ser processado.

    Returns:
        pd.DataFrame: DataFrame com as colunas de embeddings adicionadas.
    """
    total_start_time = time.time()  # Início do tempo total

    for col, out_col in zip(columns, output_columns):
        print(f"\nIniciando processamento para coluna: {col}")
        embeddings_list = []

        for i in range(0, len(df), block_size):
            start_idx = i
            end_idx = min(i + block_size, len(df))
            print(f"  Processando registros de índice {start_idx} até {end_idx - 1}...")
            
            # Selecionar o bloco atual
            block_texts = df.iloc[start_idx:end_idx][col].tolist()
            
            # Gerar embeddings para o bloco
            block_embeddings = create_embeddings(block_texts)
            
            # Adicionar ao resultado final
            embeddings_list.extend(block_embeddings)
        
        # Adicionar os embeddings ao DataFrame
        df[out_col] = embeddings_list
        print(f"  Finalizado processamento para coluna: {col}")

    # Fim do tempo total
    total_time = time.time() - total_start_time
    print(f"\nTempo total de processamento: {total_time:.2f} segundos.")

    return df

# Configuração de colunas de entrada e saída
output_columns = ['SBERT_Embeddings1', 'SBERT_Embeddings2', 'SBERT_Embeddings3', 'SBERT_Embeddings4']

# Processar o DataFrame em blocos
print("Início do processamento de embeddings para múltiplas colunas em blocos...")
df_portarias_mgi = process_embeddings_in_blocks(
    df_portarias_mgi, 
    columns_to_process, 
    output_columns, 
    block_size=200
)

print("Processamento concluído!")


Início do processamento de embeddings para múltiplas colunas em blocos...

Iniciando processamento para coluna: Texto
  Processando registros de índice 0 até 199...
  Processando registros de índice 200 até 399...
  Processando registros de índice 400 até 599...
  Processando registros de índice 600 até 799...
  Processando registros de índice 800 até 999...
  Processando registros de índice 1000 até 1199...
  Processando registros de índice 1200 até 1399...
  Processando registros de índice 1400 até 1599...
  Processando registros de índice 1600 até 1799...
  Processando registros de índice 1800 até 1999...
  Processando registros de índice 2000 até 2199...
  Processando registros de índice 2200 até 2399...
  Processando registros de índice 2400 até 2599...
  Processando registros de índice 2600 até 2799...
  Processando registros de índice 2800 até 2999...
  Processando registros de índice 3000 até 3199...
  Processando registros de índice 3200 até 3399...
  Processando registros de 

In [ ]:
# Salva saída parcial o arquivo completo com novas colunas
df_portarias_mgi.to_parquet(f'./saida/04_2_DOU{secao}_portarias_mgi_tratado_NER_lematizado_embeddingsSBERT.parquet', engine='pyarrow', index=False)

In [ ]:
# df_portarias_mgi.head(100).to_excel(f'./saida/04_2_DOU{secao}_portarias_mgi_tratado_NER_lematizado_embeddingsSBERT_100.xlsx', index=False)
